In [1]:
# Load Libraries
import warnings
from datetime import datetime

import tensorflow as tf
from keras import backend as K
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, ELU, Embedding, BatchNormalization, Convolution1D, MaxPooling1D, concatenate
from keras.layers.core import Dense, Dropout, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.models import model_from_json
from model_evaluator import Evaluator
from model_preprocessor import Preprocessor
from sklearn import model_selection
import time

warnings.filterwarnings("ignore")
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
K.tensorflow_backend.set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
def cnn(max_len=80, emb_dim=32, max_vocab_len=128, W_reg=regularizers.l2(1e-4)):
    """CNN model with the Keras functional API"""

    # Input
    main_input = Input(shape=(max_len,), dtype='int32', name='cnn_input')

    # Embedding layer
    emb = Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len, W_regularizer=W_reg)(main_input)
    emb = Dropout(0.2)(emb)

    def sum_1d(X):
        return K.sum(X, axis=1)

    def get_conv_layer(emb, kernel_size=5, filters=256):
        # Conv layer
        conv = Convolution1D(kernel_size=kernel_size, filters=filters, border_mode='same')(emb)
        conv = ELU()(conv)
        conv = MaxPooling1D(5)(conv)
        conv = Lambda(sum_1d, output_shape=(filters,))(conv)
        conv = Dropout(0.5)(conv)
        return conv

    # Multiple Conv Layers
    # 커널 사이즈를 다르게 한 conv
    conv1 = get_conv_layer(emb, kernel_size=2, filters=256)
    conv2 = get_conv_layer(emb, kernel_size=3, filters=256)
    conv3 = get_conv_layer(emb, kernel_size=4, filters=256)
    conv4 = get_conv_layer(emb, kernel_size=5, filters=256)

    # Fully Connected Layers
    # 위 결과 합침
    merged = concatenate([conv1, conv2, conv3, conv4], axis=1)

    hidden1 = Dense(1024)(merged)
    hidden1 = ELU()(hidden1)
    hidden1 = BatchNormalization(mode=0)(hidden1)
    hidden1 = Dropout(0.5)(hidden1)

    hidden2 = Dense(256)(hidden1)
    hidden2 = ELU()(hidden2)
    hidden2 = BatchNormalization(mode=0)(hidden2)
    hidden2 = Dropout(0.5)(hidden2)

    hidden3 = Dense(64)(hidden2)
    hidden3 = ELU()(hidden3)
    hidden3 = BatchNormalization(mode=0)(hidden3)
    hidden3 = Dropout(0.5)(hidden3)

    # Output layer (last fully connected layer)
    # 마지막 클래스 결정하는 layer
    output = Dense(1, activation='sigmoid', name='cnn_output')(hidden3)

    model = Model(input=[main_input], output=[output])

    return model

In [3]:
# Load data using model preprocessor
#x_data, y_data = Preprocessor.load_data_binary(10000)
#x_train, x_test, y_train, y_test = model_selection.train_test_split(x_data, y_data, test_size=0.2, random_state=33)
x_train, x_test, y_train, y_test = Preprocessor.load_data_binary(10000)

# Define Deep Learning Model
model = cnn()

''' Training phrase '''
epochs = 15
batch_size = 64
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam, loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.BinaryAccuracy(),
                       Evaluator.precision, Evaluator.recall, Evaluator.fmeasure])

history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.11)

x_train:  (8000, 80)
y_train:  (8000, 1)
x_test:  (2000, 80)
y_test:  (2000, 1)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 7120 samples, validate on 880 samples
Epoch 1/15
7120/7120 [==============================] - 7s 1ms/step - loss: 0.9282 - accuracy: 0.5549 - binary_accuracy: 0.5404 - precision: 0.4953 - recall: 0.5526 - fmeasure: 0.5170 - val_loss: 1.0121 - val_accuracy: 0.2852 - val_binary_accuracy: 0.5381 - val_precision: 1.0000 - val_recall: 0.2857 - val_fmeasure: 0.4425
Epoch 2/15
7120/7120 [==============================] - 6s 885us/step - loss: 0.6977 - accuracy: 0.6656 - binary_accuracy: 0.5576 - precision: 0.6137 - recall: 0.6538 - fmeasure: 0.6268 - val_loss: 1.1913 - val_accuracy: 0.4364 - val_binary_accuracy: 0.5865 - val_precision: 1.0000 - val_recall: 0.4368 - val_fmeasure: 0.6062
Epoch 3/15
7120/7120 [==============================] - 6s 886us/step - loss: 0.4649 - accuracy: 0.8014

In [4]:
model_json = model.to_json()
with open("./saved_model/cnn.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("./saved_model/cnn.h5")